In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from proj1_helpers import load_csv_data
from functions import *
%load_ext autoreload
%autoreload 2

In [121]:
train_set = load_csv_data('Data/train.csv', sub_sample=True)

In [122]:
test_set = load_csv_data('Data/test.csv')
test_set

(array([ 1.,  1.,  1., ...,  1.,  1.,  1.]),
 array([[-999.   ,   79.589,   23.916, ..., -999.   , -999.   ,    0.   ],
        [ 106.398,   67.49 ,   87.949, ..., -999.   , -999.   ,   47.575],
        [ 117.794,   56.226,   96.358, ..., -999.   , -999.   ,    0.   ],
        ..., 
        [ 108.497,    9.837,   65.149, ..., -999.   , -999.   ,    0.   ],
        [  96.711,   20.006,   66.942, ..., -999.   , -999.   ,   30.863],
        [  92.373,   80.109,   77.619, ..., -999.   , -999.   ,    0.   ]]),
 array([350000, 350001, 350002, ..., 918235, 918236, 918237]))

In [ ]:
train_set_pd = pd.read_csv('Data/train.csv')

In [123]:
test_set_pd = pd.read_csv('Data/test.csv')

Separate the data depending on the columns used. And clean the first column by replacing -999 by the mean of the clean values.

In [ ]:
train_set_pd_0 = train_set_pd[train_set_pd.PRI_jet_num == 0].drop(['DER_deltaeta_jet_jet', 'DER_mass_jet_jet', 'DER_prodeta_jet_jet', 'DER_lep_eta_centrality', 'PRI_jet_leading_pt', 'PRI_jet_leading_eta', 'PRI_jet_leading_phi', 'PRI_jet_subleading_pt', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_num'], axis = 1)
train_set_pd_1 = train_set_pd[train_set_pd.PRI_jet_num == 1].drop(['DER_deltaeta_jet_jet', 'DER_mass_jet_jet', 'DER_prodeta_jet_jet', 'DER_lep_eta_centrality', 'PRI_jet_subleading_pt', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_num'], axis = 1)
train_set_pd_2 = train_set_pd[train_set_pd.PRI_jet_num.isin([2, 3])].drop(['PRI_jet_num'], axis = 1)

In [ ]:
mean_0 = get_column_mean(-999.000, train_set_pd_0, 'DER_mass_MMC')
mean_1 = get_column_mean(-999.000, train_set_pd_1, 'DER_mass_MMC')
mean_2 = get_column_mean(-999.000, train_set_pd_2, 'DER_mass_MMC')

train_set_pd_0['DER_mass_MMC'] = train_set_pd_0['DER_mass_MMC'].apply(lambda x: replace_by_mean(x, mean_0, 'DER_mass_MMC'))
train_set_pd_1['DER_mass_MMC'] = train_set_pd_1['DER_mass_MMC'].apply(lambda x: replace_by_mean(x, mean_1, 'DER_mass_MMC'))
train_set_pd_2['DER_mass_MMC'] = train_set_pd_2['DER_mass_MMC'].apply(lambda x: replace_by_mean(x, mean_2, 'DER_mass_MMC'))

Export the dataframe then we can open them with the load_csv function.

In [ ]:
train_set_pd_0.to_csv('Data/train_set_0.csv', index = False)
train_set_pd_1.to_csv('Data/train_set_1.csv', index = False)
train_set_pd_2.to_csv('Data/train_set_2.csv', index = False)

In [ ]:
train_set_0 = load_csv_data('Data/train_set_0.csv', sub_sample=True)
train_set_1 = load_csv_data('Data/train_set_1.csv', sub_sample=True)
train_set_2 = load_csv_data('Data/train_set_2.csv', sub_sample=True)

In [ ]:
x_train_0 = train_set_0[1]
x_train_1 = train_set_1[1]
x_train_2 = train_set_2[1]

y_train_0 = train_set_0[0]
y_train_1 = train_set_1[0]
y_train_2 = train_set_2[0]

In [ ]:
y_train_0 = neg_to_zero(y_train_0)
y_train_1 = neg_to_zero(y_train_1)
y_train_2 = neg_to_zero(y_train_2)

In [ ]:
y_train_0

In [ ]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    ws = [initial_w]
    w = initial_w
    for n_iter in range(max_iters):
        gradient = compute_gradient_least_square(y, tx, w)
        loss = compute_mse(y, tx, w)
        w = w - gamma*gradient
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return loss, w

In [ ]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):    
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size):
            gradient = compute_stoch_gradient(minibatch_y, minibatch_tx, w)
            loss = compute_mse(minibatch_y, minibatch_tx, w)
            new_w = w - gamma*gradient
            w = new_w
            ws.append(w)
            losses.append(loss)
            print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
            bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return losses, ws

In [ ]:
def least_squares(y, tx):
    A = tx.T@tx
    b = tx.T@y
    w = np.linalg.solve(A, b)
    loss = compute_mse(y, tx, w)
    return w, loss

In [ ]:
def ridge_regression(y, tx, lambda_):
    X = tx.T@tx
    N = X.shape[0]
    A = (X + 2*N*lambda_*np.identity(N))
    b = tx.T@y
    w = np.linalg.solve(A, b)
    loss = compute_mse(y, tx, w)
    return w, loss

In [ ]:
def logistic_regression(y, tx, initial_w, max_iters, gamma):
    loss, w = stoch_gradient_descent_log_reg(y, tx, initial_w, max_iters, gamma)
    return w, loss

In [ ]:
def reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma):
    return w, loss

**TESTS**

In [110]:
# Tests to find the right gamma
gammas = np.logspace(-5, 0)
losses = []
for gamma in gammas:
    w, loss = logistic_regression(y_train_2, x_train_2, np.zeros(x_train_2.shape[1]), 1000, gamma)
    losses.append((gamma, loss))

In [ ]:
# Tests to find the right gamma
losses.sort(key = lambda x: x[1])
losses

In [105]:
w_0, loss_0 = logistic_regression(y_train_0, x_train_0, np.zeros(x_train_0.shape[1]), 10000, 1e-5)

In [118]:
w_1, loss_1 = logistic_regression(y_train_1, x_train_1, np.zeros(x_train_1.shape[1]), 10000, 1e-5)

In [119]:
w_2, loss_2 = logistic_regression(y_train_2, x_train_2, np.zeros(x_train_2.shape[1]), 10000, 1e-5)

**TEST ON TEST SET**

In [ ]:
# function